# Нейросеть для автодополнения текстов

<div class='alert alert-info'> 

Финальное задание спринта 2. 

Задача – создать нейросеть, которая на основе начала фразы предсказывает её продолжение. 

Последовательность работы: 

1. Взять датасет, очистить его, подготовить для обучения модели.
2. Реализовать и обучить модель на основе рекуррентных нейронных сетей.
3. Замерить качество разработанной и обученной модели.
4. Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
5. Проанализировать результаты и дать рекомендации разработчикам: стоит ли использовать лёгкую модель или лучше постараться поработать с ограничениями по памяти и использовать большую предобученную.

</div>

## Загрузка библиотек, установка констант

In [238]:
import os
import random
import re

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

In [194]:
# traditionally, 
SEED = 42
# batch size
BATCH_SIZE = 512
# Train mode; if 'preliminar', just verify the code, 
# if 'final' – train the ultimate models versions. 
# When 'preliminar' mode is switched on, reduce the
# data volume to 1000 tweets. 
TRAIN_MODE = 'preliminar' 
# name of transformer model
MODEL_NAME = 'distilgpt2'

## Обработка данных

<div class='alert alert-info'>

Данные нужно 

- привести к нижнему регистру;
- удалить ссылки, упоминания, эмодзи (по необходимости);
- заменить нестандартные символы;
- токенизировать текст.

</div>

In [195]:
# set data directory
os.chdir('C:/Users/User/Yandex.Disk/DS.projects/LSTM.Praktikum/data')
# read raw text data and save to array
with open('raw_data.txt', 'r', encoding='utf-8') as file:
    raw_data = np.array(file.read().lower().splitlines())

if TRAIN_MODE == 'preliminar': 
    raw_data = raw_data[:1000]

In [201]:
# define the function for clearing and splitting of data
def split_and_clean(row): 
    # remove the mentions (@*)
    row = re.sub(r'@.*?\s', '', row)
    # remove the URLs (http or www)
    row = re.sub(r'www.*?\s', '', row)
    row = re.sub(r'http.*?\s', '', row)
    # remove emojies ('*...anything')
    row = re.sub(r'\*([^ ]+)\s', '', row)
    # remove special symbols (&*;)
    row = re.sub(r'&([^ ]+)\;', '', row)
    # remove everything except of letters and numbers
    row = re.sub(r'[^a-z0-9\s]', '', row)
    # substitute the multiple spaces to single ones
    row = re.sub(r'[\s+]', ' ', row)
    # split the strings by spaces
    row = row.split(' ')
    # remove the empty elements from lists
    row = list(filter(None, row))
    
    return(row)

raw_data = list(map(split_and_clean, raw_data))

In [ ]:
# check the result of raw data import (deliberabely without seed)
if TRAIN_MODE == 'preliminar': 
    for _ in np.random.randint(0, len(raw_data), 10): 
        print(raw_data[_])

['this', 'week', 'is', 'not', 'going', 'as', 'i', 'had', 'hoped']
['yeah', 'i', 'know', 'it', 'was', 'horrible', 'ugh', 'saddening']
['thanks', 'for', 'bursting', 'my', 'bubble']
['i', 'cant', 'take', 'this', 'heat', 'its', 'like', 'an', 'oven', 'in', 'here', 'i', 'feel', 'sick', 'nwo']
['might', 'be', 'getting', 'a', 'sore', 'throat', 'again']
['sad', 'about', 'kutner', 'being', 'killed', 'off', 'my', 'fav', 'show', 'house']
['got', 'the', 'ebay', 'blues', 'item', 'i', 'want', 'jumped', 'from', 'no', 'bidders', 'to', 'over', '100', 'in', 'an', 'hour', 'still', 'has', '3', 'hours', 'to', 'go', 'id', 'better', 'not', 'get', 'my', 'hopes', 'up']
['well', 'there', 'was', 'this', 'really', 'cool', 'part', 'where', 'i', 'wont', 'spoil', 'it']
['was', 'intending', 'to', 'finish', 'editing', 'my', '536page', 'novel', 'manuscript', 'tonight', 'but', 'that', 'will', 'probably', 'not', 'happen', 'and', 'only', '12', 'pages', 'are', 'left']
['monkeys', 'i', 'just', 'found', 'out', 'you', 'my', 't

<div class='alert alert-info'>

Поскольку по условию задания модель получает на вход 3/4 исходного текста, фразы, в которых осталось три слова и менее, следует удалить. 

</div>

In [214]:
print(f'Количество фраз до удаления слишком коротких: {len(raw_data)}.')
# drop too short phrases
raw_data = [phrase for phrase in raw_data if len(phrase) > 3]

# check the results
print(f'Количество фраз после удаления слишком коротких: {len(raw_data)}.')
if len(min(raw_data, key=len)) < 4: 
    print('Очистка от коротких фраз прошла с ошибкой.')
else: 
    print('Все короткие фразы удалены.')


Количество фраз до удаления слишком коротких: 1000.
Количество фраз после удаления слишком коротких: 935.
Все короткие фразы удалены.


In [ ]:
# save the processed (cleaned) dataset
with open('processed_data.txt', 'w+', encoding='utf-8') as file: 
    for row in raw_data:
        file.write(' '.join(row) + '\n')

In [ ]:
# add pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
if TRAIN_MODE == 'preliminar': 
    print(tokenizer)

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)


In [ ]:
# check the results of tokenization
if TRAIN_MODE == 'preliminar': 
    print((tokenizer.encode(raw_data[11], is_split_into_words=True, add_special_tokens=True, return_tensors='pt')))
    print(len(tokenizer.encode(raw_data[11], is_split_into_words=True, add_special_tokens=True, return_tensors='pt')[0]))
    print((tokenizer.tokenize(raw_data[11], is_split_into_words=True, return_tensors='pt')))
    print(len(tokenizer.tokenize(raw_data[11], is_split_into_words=True, return_tensors='pt')))
    print((raw_data[11]))
    print(len(raw_data[11]))

tensor([[ 340,  340, 9853, 4686,   74, 1521, 1312,  750, 2035,  345, 1239, 1561,
          284,  502, 7471]])
15
['Ġit', 'Ġit', 'Ġcounts', 'Ġid', 'k', 'Ġwhy', 'Ġi', 'Ġdid', 'Ġeither', 'Ġyou', 'Ġnever', 'Ġtalk', 'Ġto', 'Ġme', 'Ġanymore']
15
['it', 'it', 'counts', 'idk', 'why', 'i', 'did', 'either', 'you', 'never', 'talk', 'to', 'me', 'anymore']
14


<div class='alert alert-info'>

По условию задания, обучающая выборка 80%, валидационная и тестовая по 10%.

</div>

In [244]:
# create train, test and valid datasets
train, interhim = train_test_split(raw_data, train_size=0.8, random_state=SEED)
valid, test = train_test_split(interhim, train_size=0.5, random_state=SEED)

del interhim

# check splitting
print(f'В обучающей выборке содержится {len(train)} фраз.')
print(f'В валидационной выборке содержится {len(valid)} фраз.')
print(f'В тестовой выборке содержится {len(test)} фраз.')

В обучающей выборке содержится 748 фраз.
В валидационной выборке содержится 93 фраз.
В тестовой выборке содержится 94 фраз.


In [246]:
# save the datasets to disk
with open('train.txt', 'w+', encoding='utf-8') as file: 
    for row in train:
        file.write(' '.join(row) + '\n')
with open('valid.txt', 'w+', encoding='utf-8') as file: 
    for row in valid:
        file.write(' '.join(row) + '\n')
with open('test.txt', 'w+', encoding='utf-8') as file: 
    for row in test:
        file.write(' '.join(row) + '\n')